# Fonction custom Flow from directory

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(
      self, 
      df, 
      subset="training", 
      batch_size=64, 
      shuffle=True
    ):
        self.batch_size = batch_size
        self.df = df
        self.subset = subset
        self.indices = self.df.index.tolist()
        self.shuffle = shuffle
        self.tokenizer = tokenizer
        self.max_sequence_length = max_sequence_length
        self.encoder = encoder
        self.on_epoch_end()
        self.augmentor = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest'
        )
​
    def __len__(self):
        return len(self.indices) // self.batch_size
    
    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X, y = self.__get_data(batch)
        return X, y
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
    
    def __get_data(self, batch):
        df_batch = self.df.query("index in @batch") 
        X = self.__get_X_image(df_batch)
        y = self.__get_Y(df_batch) ?
        return X, y
    
    def __get_X_image(self, df):
        X = load_images(df) # the load_images function randomly chooses an alternate image for a given product_id
        if self.subset == "training":
            X_gen = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)
            return next(X_gen)
        else:
            return X
    
    def __get_X_text(self, df):
        X = self.tokenizer.texts_to_sequences(df["input"].values)
        X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=self.max_sequence_length)
        return X
    
    def __get_Y(self, df):
        return self.encoder.transform(np.array(df["prdtypecode"]).reshape(-1, 1)).toarray()

In [ ]:
import os
import random
import numpy as np
​
def load_image(galaxy_id):
    # target = 'pull' or target = 'tees'
    imagename = f"images/{galaxy_id}.jpg"
    img = Image.open(imagename)
    return np.array(img)
    
def load_images(mini_df):
    img_list = []
    for _, row in mini_df.iterrows():
        img = load_image(row["id"], row["target"])
        img_list.append(img) 
    return np.stack(img_list)